In [1]:
import keras
from keras.utils import np_utils
from keras import backend as K

# "th": "backend is theano", input_shape must be (channels, height, width)
# "tf": "backend is tensorflow", input_shape must be (height, width, channels)
K.set_image_dim_ordering('tf')
print("Keras version: {}".format(keras.__version__))

Using TensorFlow backend.


Keras version: 2.2.2


# Data reading / Data preprocessing

In [2]:
from keras.datasets import mnist
import numpy as np

In [3]:
# X_train, X_test: (nb_samples, 3, 32, 32) , RGB channels and data type is uint8
# Y_train, Y_test: (nb_samples,) and the value is from 0 to 9

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Parameters / Model

In [4]:
full_training = False
batch_size = 128 if full_training else 64
nb_classes = 10
nb_epoch = 12 if full_training else 1

img_rows, img_cols = 28, 28
pool_size = (2,2)
kernel_size = (3,3)

In [5]:
# (sample_num, row, col, channel)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

# one-hot encoding
# e.g. array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

## Model

In [6]:
import os
import sys
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, MaxPooling2D, Conv2D
from keras.models import save_model, load_model

In [7]:
model = Sequential()

model.add(Conv2D(64, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b1c1', input_shape=input_shape))
model.add(Conv2D(64, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b1c2'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), name='b1p1'))

model.add(Conv2D(128, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b2c1'))
model.add(Conv2D(128, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b2c2'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), name='b2p1'))

model.add(Conv2D(256, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b3c1'))
model.add(Conv2D(256, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b3c2'))
model.add(Conv2D(256, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b3c3'))
model.add(Conv2D(256, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b3c4'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), name='b3p1'))

model.add(Conv2D(512, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b4c1'))
model.add(Conv2D(512, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b4c2'))
model.add(Conv2D(512, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b4c3'))
model.add(Conv2D(512, activation='relu', kernel_size=(kernel_size[0], kernel_size[1]), padding='SAME', name='b4c4'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), name='b4p1'))

# flatten
model.add(Flatten())

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# Second Dense Layer (FC 2)
model.add(Dense(nb_classes, activation='softmax'))

## Show Model Summary

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
b1c1 (Conv2D)                (None, 28, 28, 64)        640       
_________________________________________________________________
b1c2 (Conv2D)                (None, 28, 28, 64)        36928     
_________________________________________________________________
b1p1 (MaxPooling2D)          (None, 14, 14, 64)        0         
_________________________________________________________________
b2c1 (Conv2D)                (None, 14, 14, 128)       73856     
_________________________________________________________________
b2c2 (Conv2D)                (None, 14, 14, 128)       147584    
_________________________________________________________________
b2p1 (MaxPooling2D)          (None, 7, 7, 128)         0         
_________________________________________________________________
b3c1 (Conv2D)                (None, 7, 7, 256)         295168    
__________

# Target

In [9]:
# define loss function, optimizer, and metrics
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

# load the model if it exists
modelpath = os.path.join('.','keras_model','vggnet.h5')
if os.path.isfile(modelpath):
    model = load_model(modelpath)

# Learning / Tensorboard

In [ ]:
# generate the tensorboard
# tensorboard --logdir path_to_current_dir/Graph 
tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join('.','keras_model','graph'), histogram_freq=0, write_graph=True, write_images=True)

# training the model
model.fit(x_train, y_train, \
          batch_size=batch_size, \
          epochs=nb_epoch, \
          verbose=1, \
          validation_data=(x_test, y_test), \
          shuffle=True, \
          callbacks=[tbCallBack])

## Evaluating

In [10]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Score: {}.'.format(score[0]))
print('Test Accuracy: {}.'.format(score[1]))

Test Score: 0.06625349406016466.
Test Accuracy: 0.9878.


# Prediction / Inference

In [11]:
pred = model.predict(x_test[0:2])
print(pred)
print(np.argmax(pred, axis=1))
print(y_test[0:2])

[[2.2765799e-14 3.0242362e-12 2.0169363e-12 5.5346569e-14 1.4787643e-11
  3.4063450e-14 2.1038343e-14 1.0000000e+00 3.8658308e-15 1.2533604e-11]
 [2.1750508e-11 9.3397390e-10 1.0000000e+00 6.8733001e-12 4.2146289e-13
  9.5550390e-15 1.0545298e-11 9.9958919e-10 1.0646103e-11 9.0977827e-15]]
[7 2]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


### Save the model

In [12]:
save_model(model, modelpath)